## Notebooks to make the queries to the api of the models to get the response

In [1]:
import pandas as pd
import requests
from typing import Dict
import time
import os
import json
import re

### Reading the data

In [2]:
data = pd.read_csv("../02-dataset/data/cleaned/all_questions_df_english.csv")
data

,Unnamed: 0,description,alternatives,year,subject,ground_truth,index_question,text_concat_english,text_concat_portuguese
0,0,Questão 4) A Minor Bird I have wished a bird w...,"['A) culpa por não poder cuidar do pássaro.', ...",2020,"Linguagens, códigos e suas tecnologias",D,4,Question 4) Minor Bird I Have Wished a Bird Wo...,Questão 4) A Minor Bird I have wished a bird w...
1,1,"Questão 5) Finally, Aisha finished with her cu...","['A) reforçam um padrão de beleza.', 'B) retra...",2020,"Linguagens, códigos e suas tecnologias",C,5,"Question 5) Finally, Aisha Finished with Her C...","Questão 5) Finally, Aisha finished with her cu..."
2,2,Questão 01) Pablo Pueblo Regresa un hombre en ...,['A) contrapor a individualidade de um sujeito...,2020,"Linguagens, códigos e suas tecnologias",A,6,Question 01) Pablo Pueblo returns a Hombre en ...,Questão 01) Pablo Pueblo Regresa un hombre en ...
3,3,"Questão 4) Oye, Pito, ésta es: la vida bruta d...",['A) rejeição da língua utilizada por seus ant...,2020,"Linguagens, códigos e suas tecnologias",E,9,"Question 4) Oye, Pito, Estas Es: La Life of Un...","Questão 4) Oye, Pito, ésta es: la vida bruta d..."
4,4,Questão 5) Poco después apareció en casa de El...,['A) determinação para conduzir discussões pes...,2020,"Linguagens, códigos e suas tecnologias",E,10,Question 5) Poco Después Apareció en House of ...,Questão 5) Poco después apareció en casa de El...
...,...,...,...,...,...,...,...,...,...
1001,1001,QuESTÃO 167) Em um jogo disputado em uma mesa ...,"['A) Arthur, pois a soma que escolheu é a meno...",2011,Matemática e suas tecnologias,C,172,Question 167) In a game played at a pool table...,QuESTÃO 167) Em um jogo disputado em uma mesa ...
1002,1002,QuESTÃO 168) É possível usar água ou comida pa...,"['A) 20 mL.', 'B) 24 mL.', 'C) 100 mL.', 'D) 1...",2011,Matemática e suas tecnologias,C,173,Question 168) It is possible to use water or f...,QuESTÃO 168) É possível usar água ou comida pa...
1003,1003,"QuESTÃO 171) Nos últimos cinco anos, 32 mil mu...","['A) 4 mil.', 'B) 9 mil.', 'C) 21 mil.', 'D) 3...",2011,Matemática e suas tecnologias,D,176,"Question 171) In the last five years, 32,000 w...","QuESTÃO 171) Nos últimos cinco anos, 32 mil mu..."
1004,1004,QuESTÃO 174) O setor de recursos humanos de um...,"['A) 24.', 'B) 31.', 'C) 32.', 'D) 88.', 'E) 8...",2011,Matemática e suas tecnologias,E,179,Question 174) The human resources sector of a ...,QuESTÃO 174) O setor de recursos humanos de um...


## Making the requests

In [6]:
def get_text(line : pd.Series, text_add_prompt: str, lang: str):
    """
    """
    if lang == 'pt':
        text = line['text_concat_portuguese']
    else:
        text = line['text_concat_english']
    if text_add_prompt != "":
        text += "\n" + text_add_prompt
    return text

def make_querie(url : str, text : str, temperature: float = None, stop_tokens: str = None):
    """
    """
    params = {}
    params['query'] = text
    if not(temperature is None):
        params['temperature'] = temperature
    if not(stop_tokens is None):
        params['stop_tokens'] = stop_tokens
    start_time = time.time()
    request = requests.get(url, params = params)

    return_infos = {
        'time_to_run': time.time() - start_time,
        'result': eval(request.text)['result']
    }
    
    return return_infos

def make_all_requests(
        df: pd.DataFrame, 
        models: Dict, 
        temperature: float,
        stop_tokens: str,
        path_to_save: str,
        text_add_prompt: str,
        lang: str,
        base_path: str = "http://localhost:8000",
        verbose: bool = True
    ):
    """
    """
    
    try:
        os.mkdir(path_to_save)
    except:
        pass

    for model,url in models.items():
        
        for index,line in df.iterrows():
            name_arq = f"{line['year']}-{line['index_question']}-{model}.json"
            full_path_arq = f"{path_to_save}/{name_arq}"
            
            # if alredy process, not process
            if os.path.exists(full_path_arq):
                continue
            
            # otherwise compute
            url_req = base_path + url
            
            result = make_querie(
                url = url_req,
                text = get_text(line, text_add_prompt, lang),
                temperature = temperature,
                stop_tokens = stop_tokens
            )
            
            with open(full_path_arq, 'w') as file:
                file.write(json.dumps(result,indent=2,ensure_ascii=False))
            
            if verbose:
                print(f"--> {model} || {index} || {result}")

        print(f"Run the model {model}")
    

In [7]:
models = {
    "llama_7b":"/models/llama/7b",
    "llama_13b":"/models/llama/13b",
    "alpaca_7b": "/models/alpaca/7b",
    "alpaca_13b": "/models/alpaca/13b",
    "koala_7b": "/models/koala/7b",
    "koala_13b": "/models/koala/13b",
    "vicuna_7b": "/models/vicuna/7b",
    "vicuna_13b": "/models/vicuna/13b"
}

### First experiment. Only concatenating the text and seding to the model and collecting the response
- PT BR

In [13]:
PATH_SAVE_DATA = "data/01-only-question"
LANG = 'pt'
TEXT_ADD_PROMPT = ""

make_all_requests(
    df = data,
    models = models,
    temperature = 0.1,
    stop_tokens = "pergunta:,</s>,resposta:,resposta,\n",
    text_add_prompt = TEXT_ADD_PROMPT,
    path_to_save = PATH_SAVE_DATA,
    lang = LANG
)

Run the model llama_7b
Run the model llama_13b
Run the model alpaca_7b
Run the model alpaca_13b
Run the model koala_7b
Run the model koala_13b
Run the model vicuna_7b
Run the model vicuna_13b


### Reading the data processed

In [19]:
def get_response(
    path_jsons: str,
    dataframe: pd.DataFrame,
    models: Dict
):
    """
    """
    for name, _ in models.items():
        aux = []
        for index,line in dataframe.iterrows():
            name_arq = f"{line['year']}-{line['index_question']}-{name}.json"
            
            with open(f"{path_jsons}/{name_arq}",'r') as file:
                json_response = json.loads(file.read())
            
            aux.append(json_response)
        dataframe[f'{name}_response'] = aux
            
    return dataframe

data_results = get_response(
    path_jsons = PATH_SAVE_DATA,
    dataframe = data.copy(),
    models = models
)

In [20]:
data_results

,description,alternatives,year,subject,ground_truth,index_question,llama_7b_response,llama_13b_response,alpaca_7b_response,alpaca_13b_response,koala_7b_response,koala_13b_response,vicuna_7b_response,vicuna_13b_response
0,Questão 4) A Minor Bird I have wished a bird w...,"['A) culpa por não poder cuidar do pássaro.', ...",2020,"Linguagens, códigos e suas tecnologias",D,4,"{'time_to_run': 11.497793912887573, 'result': ...","{'time_to_run': 22.216389179229736, 'result': ...","{'time_to_run': 13.731350898742676, 'result': ...","{'time_to_run': 38.202540159225464, 'result': ...","{'time_to_run': 18.024755716323853, 'result': ...","{'time_to_run': 39.479053258895874, 'result': ...","{'time_to_run': 17.889060020446777, 'result': ...","{'time_to_run': 49.880011796951294, 'result': ..."
1,"Questão 5) Finally, Aisha finished with her cu...","['A) reforçam um padrão de beleza.', 'B) retra...",2020,"Linguagens, códigos e suas tecnologias",C,5,"{'time_to_run': 24.429720401763916, 'result': ...","{'time_to_run': 46.84222722053528, 'result': '...","{'time_to_run': 28.91129755973816, 'result': '...","{'time_to_run': 58.18705463409424, 'result': '...","{'time_to_run': 31.245052099227905, 'result': ...","{'time_to_run': 58.128968238830566, 'result': ...","{'time_to_run': 30.32240605354309, 'result': '...","{'time_to_run': 88.94173121452332, 'result': '..."
2,Questão 01) Pablo Pueblo Regresa un hombre en ...,['A) contrapor a individualidade de um sujeito...,2020,"Linguagens, códigos e suas tecnologias",A,6,"{'time_to_run': 17.292645692825317, 'result': ...","{'time_to_run': 33.15575456619263, 'result': '...","{'time_to_run': 21.632791996002197, 'result': ...","{'time_to_run': 47.687862396240234, 'result': ...","{'time_to_run': 20.38312077522278, 'result': '...","{'time_to_run': 48.10079622268677, 'result': '...","{'time_to_run': 25.049573183059692, 'result': ...","{'time_to_run': 67.6659996509552, 'result': ' ..."
3,"Questão 4) Oye, Pito, ésta es: la vida bruta d...",['A) rejeição da língua utilizada por seus ant...,2020,"Linguagens, códigos e suas tecnologias",E,9,"{'time_to_run': 15.333454847335815, 'result': ...","{'time_to_run': 29.48138928413391, 'result': '...","{'time_to_run': 18.078598260879517, 'result': ...","{'time_to_run': 39.06972050666809, 'result': '...","{'time_to_run': 18.10067129135132, 'result': '...","{'time_to_run': 39.59393572807312, 'result': '...","{'time_to_run': 20.068461656570435, 'result': ...","{'time_to_run': 58.02386522293091, 'result': '..."
4,Questão 5) Poco después apareció en casa de El...,['A) determinação para conduzir discussões pes...,2020,"Linguagens, códigos e suas tecnologias",E,10,"{'time_to_run': 21.91682267189026, 'result': '...","{'time_to_run': 41.775152921676636, 'result': ...","{'time_to_run': 23.680744409561157, 'result': ...","{'time_to_run': 51.546794414520264, 'result': ...","{'time_to_run': 25.72270917892456, 'result': '...","{'time_to_run': 53.0454466342926, 'result': ' ...","{'time_to_run': 27.125627517700195, 'result': ...","{'time_to_run': 76.35341024398804, 'result': '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,QuESTÃO 167) Em um jogo disputado em uma mesa ...,"['A) Arthur, pois a soma que escolheu é a meno...",2011,Matemática e suas tecnologias,C,172,"{'time_to_run': 20.472726106643677, 'result': ...","{'time_to_run': 38.94875693321228, 'result': '...","{'time_to_run': 135.01702332496643, 'result': ...","{'time_to_run': 45.95291447639465, 'result': '...","{'time_to_run': 26.507246732711792, 'result': ...","{'time_to_run': 51.06923484802246, 'result': '...","{'time_to_run': 26.297470808029175, 'result': ...","{'time_to_run': 86.44696116447449, 'result': '..."
1002,QuESTÃO 168) É possível usar água ou comida pa...,"['A) 20 mL.', 'B) 24 mL.', 'C) 100 mL.', 'D) 1...",2011,Matemática e suas tecnologias,C,173,"{'time_to_run': 21.011848211288452, 'result': ...","{'time_to_run': 40.99883484840393, 'result': '...","{'time_to_run': 54.77617812156677, 'result': '...","{'t

## Defining the heuristics to collect the response of the queries

In [69]:
def first_caracter(text: str):
    """
    """
    text = text.strip()
    if len(text) > 1 and text[0].isupper() and not(text[1].isalnum()) and text[0] in 'ABCDE':
        return text[0]
    if len(text) == 1 and text[0] in 'ABCDE':
        return text[0]

    return None

def identify_alternative_mid_text(text: str):
    """
    """
    regex1 = " [A-E] ?\)? "
    matches1 = re.findall(regex1,text)
    
    regex2 = " [A-E]\)?"
    matches2 = re.findall(regex2,text)
    
    matches = matches1 + matches2
    
    if len(matches) == 1:
        options = "ABCDE"
        for o in options:
            if o in matches[0]:
                return o
    return None


def run_identify_alternative_result(
    dataframe: pd.DataFrame
):
    """
    """
    heuristics = [
        first_caracter,
        identify_alternative_mid_text
    ]
    
    columns = [c for c in dataframe.columns if c.endswith("_response")]
    
    for column in columns:
        results = []
        for index,line in dataframe.iterrows():
            
            result = None
            for heuristic in heuristics:
                out = heuristic(line[column]['result'])
                if not(out is None):
                    result = out
                    break
            
            if not(result is None):
                results.append(result)
            else:
                results.append('-')
        dataframe[column.replace("_response","_prediction")] = results
    return dataframe
    
    

In [70]:
df_predictions = run_identify_alternative_result(data_results)
df_predictions

,description,alternatives,year,subject,ground_truth,index_question,llama_7b_response,llama_13b_response,alpaca_7b_response,alpaca_13b_response,...,vicuna_7b_response,vicuna_13b_response,llama_7b_prediction,llama_13b_prediction,alpaca_7b_prediction,alpaca_13b_prediction,koala_7b_prediction,koala_13b_prediction,vicuna_7b_prediction,vicuna_13b_prediction
0,Questão 4) A Minor Bird I have wished a bird w...,"['A) culpa por não poder cuidar do pássaro.', ...",2020,"Linguagens, códigos e suas tecnologias",D,4,"{'time_to_run': 11.497793912887573, 'result': ...","{'time_to_run': 22.216389179229736, 'result': ...","{'time_to_run': 13.731350898742676, 'result': ...","{'time_to_run': 38.202540159225464, 'result': ...",...,"{'time_to_run': 17.889060020446777, 'result': ...","{'time_to_run': 49.880011796951294, 'result': ...",D,A,A,A,A,C,A,B
1,"Questão 5) Finally, Aisha finished with her cu...","['A) reforçam um padrão de beleza.', 'B) retra...",2020,"Linguagens, códigos e suas tecnologias",C,5,"{'time_to_run': 24.429720401763916, 'result': ...","{'time_to_run': 46.84222722053528, 'result': '...","{'time_to_run': 28.91129755973816, 'result': '...","{'time_to_run': 58.18705463409424, 'result': '...",...,"{'time_to_run': 30.32240605354309, 'result': '...","{'time_to_run': 88.94173121452332, 'result': '...",B,A,A,C,A,C,C,C
2,Questão 01) Pablo Pueblo Regresa un hombre en ...,['A) contrapor a individualidade de um sujeito...,2020,"Linguagens, códigos e suas tecnologias",A,6,"{'time_to_run': 17.292645692825317, 'result': ...","{'time_to_run': 33.15575456619263, 'result': '...","{'time_to_run': 21.632791996002197, 'result': ...","{'time_to_run': 47.687862396240234, 'result': ...",...,"{'time_to_run': 25.049573183059692, 'result': ...","{'time_to_run': 67.6659996509552, 'result': ' ...",D,A,A,A,A,A,A,B
3,"Questão 4) Oye, Pito, ésta es: la vida bruta d...",['A) rejeição da língua utilizada por seus ant...,2020,"Linguagens, códigos e suas tecnologias",E,9,"{'time_to_run': 15.333454847335815, 'result': ...","{'time_to_run': 29.48138928413391, 'result': '...","{'time_to_run': 18.078598260879517, 'result': ...","{'time_to_run': 39.06972050666809, 'result': '...",...,"{'time_to_run': 20.068461656570435, 'result': ...","{'time_to_run': 58.02386522293091, 'result': '...",E,A,A,A,A,A,C,D
4,Questão 5) Poco después apareció en casa de El...,['A) determinação para conduzir discussões pes...,2020,"Linguagens, códigos e suas tecnologias",E,10,"{'time_to_run': 21.91682267189026, 'result': '...","{'time_to_run': 41.775152921676636, 'result': ...","{'time_to_run': 23.680744409561157, 'result': ...","{'time_to_run': 51.546794414520264, 'result': ...",...,"{'time_to_run': 27.125627517700195, 'result': ...","{'time_to_run': 76.35341024398804, 'result': '...",E,A,A,D,A,A,D,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,QuESTÃO 167) Em um jogo disputado em uma mesa ...,"['A) Arthur, pois a soma que escolheu é a meno...",2011,Matemática e suas tecnologias,C,172,"{'time_to_run': 20.472726106643677, 'result': ...","{'time_to_run': 38.94875693321228, 'result': '...","{'time_to_run': 135.01702332496643, 'result': ...","{'time_to_run': 45.95291447639465, 'result': '...",...,"{'time_to_run': 26.297470808029175, 'result': ...","{'time_to_run': 86.44696116447449, 'result': '...",D,D,-,D,A,A,E,B
1002,QuESTÃO 168) É possível usar água ou comida pa...,"['A) 20 mL.', 'B) 24 mL.', 'C) 100 mL.', 'D) 1...",2011,Matemática e suas tecnologias,C,173,"{'time_to_run': 21.011848211288452, 'result': ...","{'time_to_run': 40.99883484840393, 'result': '...","{'time_to_run': 54.77617812156677, 'result': '...","{'time_to_run': 45.522027254104614, 'result': ...",...,"{'time_to_run': 24.036888360977173, 'result': ...","{'time_to_run': 71.67180371284485, 'result': '...",C,-,A,A,A,A,D,A
1003,"QuESTÃO 171) Nos últimos cinco anos, 32 mil mu...","['A) 4 mil.', 'B) 9 mil.', 'C) 21 mil.', 'D) 3...",2011,Matemática e suas tecnologias,D,176,"{'time_to_run': 10

### How much of the all predictions where detected with the label

In [71]:
def compute_percentage_predictions(
    dataframe: pd.DataFrame
):
    """
    """
    
    columns = [c for c in dataframe.columns if c.endswith("_prediction")]
    
    d = dataframe[columns]
    d = d == '-'
    a,b = d.shape
    d = d.sum().sum()
    
    return 1 - (d / (a*b))

c = compute_percentage_predictions(df_predictions)
print(f"Coverage predictions: {c}")

Coverage predictions: 0.9917992047713717


In [72]:
def verify_results_text(
    dataframe: pd.DataFrame
):
    """
    """
    
    columns = [c for c in dataframe.columns if c.endswith("_prediction")]
    
    for column in columns:
        
        for index,line in dataframe.iterrows():
            
            if line[column] == '-':
                text = line[column.replace("_prediction","_response")]['result']
                print(f"--> {text}")
verify_results_text(df_predictions)

-->  77) Vemos que toda cidade é uma espécie de comunidade, e toda comunidade se forma com vistas a algum bem, pois todas as ações de todos os homens são praticadas com vistas ao que lhe parece um bem; se todas as comunidades visam algum bem, é evidente que a mais importante de todas elas e que inclui todas as outras tem mais que todas este objetivo e visa ao mais importante de todos os bens. ARISTÓTELES. Política. Brasília: UnB,1988.
-->  1) A) Possibilidade da liberdade e obrigação da ação.
--> 
-->  15
-->  1) A) 2) B) 3) C) 4) D) 5) E) 6) F) 7) G) 8) H) 9) I) 10) J) 11) K) 12) L) 13) M) 14) N) 15) O) 16) P) 17) Q) 18) R) 19) S) 20) T) 21) U) 22) V) 23) W) 24) X) 25) Y) 26) Z) 27) A) 28) B) 29) C) 30) D) 31) E) 32) F) 33) G) 34) H) 35) I) 36) J) 37) K) 38) L) 39) M) 40) N) 41) O) 42) P) 43) Q) 44) R) 45) S)
-->  73) A 
-->  20%
-->  QUESTÃO 06) O eclipse Quando Fray Bartolomé Arrazola se sentiu perdido aceitou que nada poderia salvarlo. A selva poderosa de Guatemala o había apresado

-->  01, 02, 03, 04, 05
-->  2007-08-01
-->  ver tópicos anteriores (91 a 120).


## Computing the accuracy of the models

In [100]:
def accuracy(ground_truth, prediction):
    """
    """
    
    eq = ground_truth == prediction
    
    return eq.sum() / len(eq)

def compute_acc_by_year(
    df_predictions: pd.DataFrame
):
    """
    """
    years = list(set(df_predictions['year']))
    years.sort()
    
    columns = [c for c in df_predictions.columns if c.endswith('_prediction')]
    
    list_output = []
    
    
    for column in columns:
        model = column.replace("_prediction","")
        aux = [model]
        for year in years:
            
            df_aux = df_predictions[df_predictions['year'] == year]
            
            aux.append(accuracy(df_aux['ground_truth'],df_aux[column]))
        list_output.append(aux)
            
    return pd.DataFrame(list_output,columns=['model']+years)
    
    

In [101]:
accuracy(df_predictions['ground_truth'],df_predictions['vicuna_13b_prediction'])

0.3866799204771372

## Accuracy by year

In [102]:
compute_acc_by_year(df_predictions)

,model,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2022
0,llama_7b,0.236559,0.21,0.255319,0.268657,0.191919,0.194690,0.191304,0.327273,0.300000,0.282828,0.114754
1,llama_13b,0.193548,0.25,0.234043,0.253731,0.151515,0.150442,0.234783,0.145455,0.163636,0.151515,0.278689
2,alpaca_7b,0.182796,0.24,0.138298,0.253731,0.181818,0.230088,0.165217,0.200000,0.200000,0.171717,0.213115
3,alpaca_13b,0.387097,0.32,0.319149,0.373134,0.393939,0.353982,0.382609,0.327273,0.309091,0.373737,0.475410
4,koala_7b,0.193548,0.26,0.138298,0.179104,0.181818,0.194690,0.165217,0.109091,0.163636,0.181818,0.245902
5,koala_13b,0.311828,0.36,0.319149,0.268657,0.323232,0.256637,0.304348,0.272727,0.281818,0.343434,0.377049
6,vicuna_7b,0.322581,0.32,0.287234,0.358209,0.313131,0.283186,0.356522,0.309091,0.345455,0.393939,0.344262
7,vicuna_13b,0.408602,0.33,0.340426,0.417910,0.393939,0.398230,0.339130,0.454545,0.400000,0.424242,0.393443


## Accuracy by knowlege area

In [103]:
def compute_acc_by_knowlege_area(
    df_predictions: pd.DataFrame
):
    """
    """
    subjects = list(set(df_predictions['subject']))
    
    columns = [c for c in df_predictions.columns if c.endswith('_prediction')]
    
    list_output = []
    
    
    for column in columns:
        model = column.replace("_prediction","")
        aux = [model]
        for sub in subjects:
            
            df_aux = df_predictions[df_predictions['subject'] == sub]
            
            aux.append(accuracy(df_aux['ground_truth'],df_aux[column]))
        list_output.append(aux)
            
    return pd.DataFrame(list_output,columns=['model']+subjects)

In [104]:
compute_acc_by_knowlege_area(df_predictions)

,model,"Linguagens, códigos e suas tecnologias",Matemática e suas tecnologias,Ciências da natureza e suas tecnologias,Ciências humanas e suas tecnologias
0,llama_7b,0.226148,0.217949,0.244541,0.236686
1,llama_13b,0.208481,0.147436,0.227074,0.192308
2,alpaca_7b,0.215548,0.141026,0.192140,0.207101
3,alpaca_13b,0.395760,0.198718,0.336245,0.426036
4,koala_7b,0.204947,0.141026,0.196507,0.177515
5,koala_13b,0.363958,0.134615,0.288210,0.360947
6,vicuna_7b,0.310954,0.205128,0.296943,0.426036
7,vicuna_13b,0.462898,0.192308,0.301310,0.470414


## Overall accuracy

In [116]:
def compute_acc_overall(
    df_predictions: pd.DataFrame
):
    """
    """
    subjects = list(set(df_predictions['subject']))
    
    columns = [c for c in df_predictions.columns if c.endswith('_prediction')]
    
    list_output = []
    
    
    for column in columns:
        list_output.append(accuracy(df_predictions['ground_truth'],df_predictions[column]))
            
    return pd.DataFrame([list_output],columns=[c.replace("_prediction","") for c in columns ])
    
    

In [117]:
compute_acc_overall(df_predictions)

,llama_7b,llama_13b,alpaca_7b,alpaca_13b,koala_7b,koala_13b,vicuna_7b,vicuna_13b
0,0.232604,0.197813,0.195825,0.361829,0.183897,0.310139,0.33002,0.38668
